In [1]:
!pip install langchain
!pip install -U duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 816.1/816.1 kB 9.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 398.8/398.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 25.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.7/83.7 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 23.1
    Uninstalling packaging-23.1:
      Successfully uninstalled packaging-23.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 6.3 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 16.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: nest-asyncio
    Found existing installation: nest-asyncio 1.5.6
    Uninstalling nest-asyncio-1.5.6:
      Successfully uninstalled nest-asyncio-1.5.6
  Attempting uninstall: lxml
    Found existing installation: lxml 4.9.3
    Uninstalling lxml-4.9.3:
      Successfully uninstalled lxml-4.9.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.1.5 requires pyqt5<5.13, which is not installed.
spyder 5.1.5 requires pyqtwebengine<5.13, which is not installed.


In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

In [3]:
from langchain.tools import DuckDuckGoSearchResults

In [4]:
import json
search = DuckDuckGoSearchResults()
provided_string = search.run("Stanislaus State Basketball Roster")
# Cleaning and splitting the string to prepare for dictionary conversion
cleaned_string = provided_string.strip('[]')
snippets = cleaned_string.split('], [')

# Parsing snippets into a dictionary
direct_roster_dict = {}
for snippet in snippets:
    # Extracting link and content from each snippet
    parts = snippet.rsplit(", link: ", 1)  # Split from the right to ensure we capture the last 'link: ' as the split point
    if len(parts) == 2:
        content, link = parts
        direct_roster_dict[link] = content

list(direct_roster_dict.keys())[0]

'https://warriorathletics.com/sports/mens-basketball/roster'

In [5]:
def extract_website_prefix(url):
    # Find the position of ".com"
    com_index = url.find(".com")
    # Extract everything up to and including ".com"
    if com_index != -1:
        prefix = url[:com_index + 4]  # Adding 4 to include ".com" itself
        return prefix
    else:
        return "Not a .com URL"

In [6]:
def get_athletics_website(team):
    search_string = f'{team} Basketball Roster'
    search = DuckDuckGoSearchResults()
    search_results_string = search.run(search_string)
    cleaned_string = search_results_string.strip('[]')
    snippets = cleaned_string.split('], [')

    direct_roster_dict = {}
    for snippet in snippets:
        parts = snippet.rsplit(", link: ", 1)
        if len(parts) == 2:
            content, link = parts
            direct_roster_dict[link] = content

    website = extract_website_prefix(list(direct_roster_dict.keys())[0])
    return website

In [7]:
get_athletics_website("Wheeling University")

'https://wucardinals.com'

In [8]:
def individual_overall_table(url):

    # Fetch the webpage
    response = requests.get(url)

    # Ensure the request was successful
    if response.status_code == 200:
        # Parse the webpage with BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # ID of the section you want to extract
        section_id = 'individual-overall'

        # Find the section by ID
        section = soup.find(id=section_id)

        if section:
            # Get the HTML content of the section
            section_html = str(section)
        else:
            print(f"Section with ID {section_id} not found.")
    else:
        print(f"Failed to fetch the webpage. Status code: {response.status_code}")

    player_names_stats = {}

    player_names_raw = section.find_all('a', class_='hide-on-medium-down')
    player_stats_raw = section.find_all('td', class_='text-center')

    for i in range(len(player_names_raw)):
        player_names_stats[player_names_raw[i].text] = {player['data-label']: player.text for player in player_stats_raw[(i*24):(24-i)+(i*25)]}

    return pd.DataFrame.from_dict(player_names_stats, orient='index')

In [9]:
def get_player_names(url):

    # Fetch the webpage
    response = requests.get(url)

    # Ensure the request was successful
    if response.status_code == 200:
        # Parse the webpage with BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # ID of the section you want to extract
        section_class = 'sidearm-roster-player-name'

        # Find the section by ID
        section = soup.find_all(class_=section_class)

        if section:
            # Get the HTML content of the section
            section_html = str(section)
        else:
            print(f"Section with ID {section_class} not found.")
    else:
        print(f"Failed to fetch the webpage. Status code: {response.status_code}")

    bio_links = soup.find_all('a', {'aria-label': lambda L: L and 'View Full Bio' in L})

    # Extract player names from the 'aria-label' attribute
    player_names = [link['aria-label'].replace(' - View Full Bio', '') for link in bio_links]

    return list(dict.fromkeys(player_names))

In [10]:
def extract_position_before_space(pos):
    # Use regex to find the part before a long space
    match = re.search(r'^([\w/]+)', pos)
    if match:
        return match.group(1)
    return pos

In [11]:
def get_player_positions(url):
    # Fetch the webpage
    response = requests.get(url)

    # Ensure the request was successful
    if response.status_code == 200:
        # Parse the webpage with BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # ID of the section you want to extract
        section_class = 'sidearm-roster-player-position'

        # Find the section by ID
        section = soup.find_all(class_=section_class)

        if section:
            # Get the HTML content of the section
            section_html = str(section)
        else:
            print(f"Section with ID {section_class} not found.")
    else:
        print(f"Failed to fetch the webpage. Status code: {response.status_code}")

    position_list = []
    for sect in section:
        position_list.append(sect.find('span', class_='text-bold').text.strip().replace('\r', '').replace('\n', '').replace('\t', ''))

    return [extract_position_before_space(pos).replace("Guard", "G").replace("Power Forward", "PF").replace("Forward", "F").replace("Foward","C").replace("Center", "C") for pos in position_list]

In [12]:
def data_table(individual_url, roster_url):
    df = individual_overall_table(individual_url)
    df.index = df.index.map(lambda x: ' '.join(x.split(', ')[::-1]))

    positions = get_player_positions(roster_url)
    players = get_player_names(roster_url)[:len(positions)]
    player_position_dict = dict(zip(players, positions))

    positions_df = pd.DataFrame(list(player_position_dict.items()), columns=['Player', 'Position'])
    positions_df.set_index('Player', inplace=True)

    df = df.merge(positions_df, left_index=True, right_index=True, how='right')

    df.fillna(0, inplace=True)

    return df

In [13]:
def validate_website(url, url_type, check_type='abbr'):
    try:
        response = requests.get(url, allow_redirects=True)
        final_url = response.url  # Get the final URL after redirection

        # Check if a redirect occurred by examining the response history
        if response.history or response.status_code in range(300, 405):
            if url_type == "roster":
                if check_type == 'abbr':
                    new_url = url[:-4] + url[-2:]
                    return validate_website(new_url, 'roster', 'year')
                elif check_type == 'year':
                    new_url = url[:-7] + '20' + url[-2:]
                    return validate_website(new_url, 'roster', None)
            elif url_type == "individual_overall":
                if check_type == 'abbr':
                    new_url = url[:-15] + url[-13:]
                    return validate_website(new_url, 'individual_overall', 'year')
                elif check_type == 'year':
                    new_url = url[:-18] + '20' + url[-13:]
                    return validate_website(new_url, 'individual_overall', None)
        elif response.status_code == 200:
            #print(f"Website exists and is reachable: {final_url}")
            return final_url
        return 'Website not found'
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

In [14]:
def team_data(team):
    df = pd.DataFrame()
    website = get_athletics_website(team)
    years = ["2018-2019", "2019-2020", "2020-2021", "2021-2022", "2022-2023"]
    roster_suffix = '/sports/mens-basketball/roster/'
    individual_overall_suffix = '/sports/mens-basketball/stats/xxxYEARxxx#individual'
    for year in years:
        individual_overall_website = validate_website(website + individual_overall_suffix.replace("xxxYEARxxx", year), 'individual_overall')
        roster_website = validate_website(website + roster_suffix + year, 'roster')
        if individual_overall_website is None or roster_website is None:
            print(team, year, "Failed")
            continue
        else:
            new_df = data_table(individual_overall_website, roster_website)
            new_df['Team'] = team
            new_df['Year'] = year
            df = df.append(new_df)
            print(team, year, "Success")
    return df

In [15]:
def data_all(team_list):
    df = pd.DataFrame()
    for team in team_list:
        try:
            new_df = team_data(team)
            df = df.append(new_df)  # Optionally reset index
        except Exception as e:
            print(f"Failed to process team {team}: {e}")
            continue  # This will skip the current iteration and move to the next team
    return df

In [17]:
d2_schools = pd.read_csv("d2_basketball_schools.csv")
d2_schools

,Name,Conference,Division,Reclass Division,Public/Private,HBCU,State
0,Academy of Art University,Pacific West Conference,II,NaN,Private,No,CA
1,Adams State University,Rocky Mountain Athletic Conference,II,NaN,Public,No,CO
2,Adelphi University,Northeast-10 Conference,II,NaN,Private,No,NY
3,Albany State University (Georgia),Southern Intercol. Ath. Conf.,II,NaN,Public,Yes,GA
4,Allen University,Southern Intercol. Ath. Conf.,II,NaN,Private,Yes,SC
...,...,...,...,...,...,...,...
309,Wilmington University (Delaware),Central Atlantic Collegiate Conference,II,NaN,Private,No,DE
310,Wingate University,South Atlantic Conference,II,NaN,Private,No,NC
311,Winona State University,Northern Sun Intercollegiate Conference,II,NaN,Public,No,MN
312,Winston-Salem State University,Central Intercollegiate Athletic Association,II,NaN,Public,Yes,NC


In [33]:
players=pd.read_csv('data (3).csv')
players

,Player,GP,GS,MIN,MIN/G,FGM,FGA,FG%,3PT,3PTA,...,REB,REB/G,PF,AST,TO,STL,BLK,Position,Team,Year
0,Andrija Blatancic,23,19,698,30.3,162,299,0.542,2,18,...,202,8.8,68,24,61,21,6,F,Academy of Art University,2018-2019
1,Dante Williams,28,1,409,14.6,52,109,0.477,6,18,...,40,1.4,51,22,34,13,1,G,Academy of Art University,2018-2019
2,Chance Maxwell,14,0,61,4.4,2,4,0.500,0,0,...,3,0.2,7,6,1,2,0,G,Academy of Art University,2018-2019
3,Cole Morgan,23,9,373,16.2,35,96,0.365,2,8,...,86,3.7,56,28,28,22,2,G,Academy of Art University,2018-2019
4,Cobe Williams,28,27,742,26.5,86,206,0.417,47,134,...,36,1.3,62,91,62,26,1,G,Academy of Art University,2018-2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15638,Ahmir Crum,0,0,0,0.0,0,0,0.000,0,0,...,0,0.0,0,0,0,0,0,G,Young Harris College,2022-2023
15639,Alfonzo Hilliard,0,0,0,0.0,0,0,0.000,0,0,...,0,0.0,0,0,0,0,0,G,Young Harris College,2022-2023
15640,Avery Jones,0,0,0,0.0,0,0,0.000,0,0,...,0,0.0,0,0,0,0,0,G,Young Harris College,2022-2023
15641,Griffin Neville,0,0,0,0.0,0,0,0.000,0,0,...,0,0.0,0,0,0,0,0,G,Young Harris College,2022-2023


In [61]:
#Similarity for players
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics.pairwise import cosine_similarity

def find_similar_players(player_data, k=5):
    
    numeric_cols = player_data.select_dtypes(include=['number']).columns
    categorical_cols = ['Position','Team','Year']
    
    # Apply transformations to numeric and categorical columns
    numeric_transformer = StandardScaler()
    categorical_transformer = OneHotEncoder()

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_cols),
            ('cat', categorical_transformer, categorical_cols)
        ])
    
    # Fit the preprocessor
    preprocessor.fit(player_data.drop(columns=['Player']))
    
    # Transform the data
    transformed_data = preprocessor.transform(player_data.drop(columns=['Player']))

    # Calculate cosine similarity for numeric columns
    numeric_similarity_matrix = cosine_similarity(transformed_data[:, :len(numeric_cols)], transformed_data[:, :len(numeric_cols)])

    # Calculate similarity for categorical columns (0 for same, 1 for different)
    categorical_similarity_matrix = (transformed_data[:, len(numeric_cols):] != transformed_data[:, len(numeric_cols):]).astype(int)

    # Combine similarity scores for numeric and categorical columns
    similarity_matrix = np.concatenate((numeric_similarity_matrix, categorical_similarity_matrix), axis=1)

    # Get the similarity scores for the new player
    new_player_similarity = similarity_matrix[:, 0]

    # Get the indices of the top k most similar players
    top_k_indices = new_player_similarity.argsort()[::-1][:k]

    # Get the names of the top k most similar players
    top_k_players = player_data.iloc[top_k_indices]['Player']

    return top_k_players

find_similar_players(players.head(10), k=2)

0          Andrija Blatancic
7    Victor Ruiz de Carranza
Name: Player, dtype: object

In [54]:
players.head(10)

,Player,GP,GS,MIN,MIN/G,FGM,FGA,FG%,3PT,3PTA,...,REB,REB/G,PF,AST,TO,STL,BLK,Position,Team,Year
0,Andrija Blatancic,23,19,698,30.3,162,299,0.542,2,18,...,202,8.8,68,24,61,21,6,F,Academy of Art University,2018-2019
1,Dante Williams,28,1,409,14.6,52,109,0.477,6,18,...,40,1.4,51,22,34,13,1,G,Academy of Art University,2018-2019
2,Chance Maxwell,14,0,61,4.4,2,4,0.500,0,0,...,3,0.2,7,6,1,2,0,G,Academy of Art University,2018-2019
3,Cole Morgan,23,9,373,16.2,35,96,0.365,2,8,...,86,3.7,56,28,28,22,2,G,Academy of Art University,2018-2019
4,Cobe Williams,28,27,742,26.5,86,206,0.417,47,134,...,36,1.3,62,91,62,26,1,G,Academy of Art University,2018-2019
5,Lake Del Mar,25,1,226,9.0,29,56,0.518,0,0,...,37,1.5,18,1,6,0,7,F/C,Academy of Art University,2018-2019
6,Nemanja Kovacevic,23,1,212,9.2,16,51,0.314,12,40,...,19,0.8,35,8,15,3,0,G,Academy of Art University,2018-2019
7,Victor Ruiz de Carranza,28,17,751,26.8,63,140,0.450,9,36,...,181,6.5,80,80,61,35,14,G,Academy of Art University,2018-2019
8,Kiel Long,28,16,538,19.2,22,75,0.293,8,41,...,57,2.0,48,32,13,7,1,G,Academy of Art University,2018-2019
9,Luka Radovic,14,4,124,8.9,11,32,0.344,0,0,...,20,1.4,17,1,15,4,4,F,Academy of Art University,2018-2019


In [18]:
d2_schools_list = list(d2_schools['Name'])
df = data_all(d2_schools_list)
df

/var/folders/nm/yzn6vc857sg_hy308zn4r2gr0000gn/T/ipykernel_31873/3117760796.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_df)


Academy of Art University 2018-2019 Success


/var/folders/nm/yzn6vc857sg_hy308zn4r2gr0000gn/T/ipykernel_31873/3117760796.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_df)


Academy of Art University 2019-2020 Success


/var/folders/nm/yzn6vc857sg_hy308zn4r2gr0000gn/T/ipykernel_31873/3117760796.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_df)


...

/var/folders/nm/yzn6vc857sg_hy308zn4r2gr0000gn/T/ipykernel_31873/3117760796.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_df)
/var/folders/nm/yzn6vc857sg_hy308zn4r2gr0000gn/T/ipykernel_31873/720814689.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_df)  # Optionally reset index


Adams State University 2022-2023 Success


/var/folders/nm/yzn6vc857sg_hy308zn4r2gr0000gn/T/ipykernel_31873/3117760796.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_df)


Adelphi University 2018-2019 Success


KeyboardInterrupt: 

In [ ]:
#